# Basic use for assessing storm climatology in a dataset
Here, we examplify usage of `huracanpy` with the dataset of TC in ERA-20C detected by the TRACK algorithm.
This is meant to show an example of workflow. Please refer to specific parts of the documentation to learn about each part (e.g. loading, plotting, etc.) in more detail.

In [ ]:
import huracanpy

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import cartopy.crs as ccrs

## Read the file
`huracanpy`'s `load` function can handle different track file types. Here, the data is available as a netcdf file.

In [ ]:
data = huracanpy.load(
    huracanpy.example_ERA20C_file
)
data.psl.attrs["units"] = "hPa"  # Fixing misspelled pressure unit
data

## Add useful information
After loading, you can add various useful information for the analysis (basin, season, category...)

In [ ]:
# Use the accessor's add_ function to add the info you want
## SSHS category
data = data.hrcn.add_saffir_simpson_category(wind_name = "wind_speed_10m")
## Presure category
data = data.hrcn.add_pressure_category(slp_name = "psl")
## Season 
data = data.hrcn.add_season()
# More info are available, in particular geographical ones, but we do not need them for this example.

In [ ]:
data

# Check the content of the file
`huracanpy` provide a coarse plotting function that you can use for checking what is in your data. 

In [ ]:
# Basic plot of the data points
data.hrcn.plot_tracks(intensity_var_name = "wind_speed_10m")

## Climatological metrics
You can compute basic statistics: frequency, TC days, ACE. Here shown as yearly averages.

In [ ]:
# Frequency (Number of track per year)
data.track_id.hrcn.nunique() / data.season.hrcn.nunique() 
# number of unique tracks / number of unique season

In [ ]:
# TCD (Accumulated duration of storms per year)
data.hrcn.get_track_duration().sum() / 24 / data.season.hrcn.nunique()
# Compute duration per track, convert to days / number of unique season

In [ ]:
# ACE per year
data = data.hrcn.add_ace(wind_name = "wind_speed_10m")
data.ace.groupby(data.season).sum().mean()
# Get ace for each point, sum by season and average over the seasons
# NB: By default, huracanpy computes ACE only for points with wind above 34 knots

## Variability
With xarray's grouping functionnalities, you can show variations of these statistics. 

In [ ]:
## Interannual

fig, axs = plt.subplots(3, sharex=True)
# Frequency
# In this case, it is easier to go through a dataframe
data.to_dataframe().groupby("season").track_id.nunique().plot(ax=axs[0]) 
axs[0].set_ylabel("Number of tracks")
# TCD
data.groupby("season").apply(lambda s: s.hrcn.get_track_duration().sum()).plot(ax=axs[1])
axs[1].set_ylabel("TC days")
# ACE
data.groupby("season").sum().ace.plot(ax=axs[2])
axs[2].set_ylabel("ACE")

for ax in axs:
    ax.set_ylim(0)
    ax.set_xlabel("")

In [ ]:
## Seasonal

gen = data.hrcn.get_gen_vals()  # Extract the point of genesis for each track
(
    gen.groupby("time.month").count().lon  # compute number of genesis points per month
    / gen.season.hrcn.nunique()  # Normalize by number of season
).plot(marker="o")  # plot

## Track statistics
You can also compute track-level statistics such as duration and lifetime maximum intensity.

In [ ]:
fig, axs = plt.subplots(1, 3, sharey=True, figsize=[15, 5])

# Duration
data.hrcn.get_track_duration().plot.hist(ax=axs[0])

# Maximum wind speed
data.wind_speed_10m.groupby(data.track_id).max().plot.hist(ax=axs[1])

# Minimum SLP
data.psl.groupby(data.track_id).min().plot.hist(ax=axs[2])

## Lifecycles

In [ ]:
# Compute times from extremum
data["time_from_slp_min"] = data.hrcn.get_time_from_apex(intensity_var_name = "psl", stat = "min")
data["time_from_wind_max"] = data.hrcn.get_time_from_apex(intensity_var_name = "wind_speed_10m",)

In [ ]:
# Plot with seaborn
fig, axs = plt.subplots(2, sharex=True)
# SLP lifecycle
sns.lineplot(data=data, x="time_from_slp_min", y="psl", ax=axs[0])
# Wind lifecycle
sns.lineplot(data=data, x="time_from_wind_max", y="wind_speed_10m", ax=axs[1])

## Track density

In [ ]:
# Density of all points
d = data.hrcn.get_density(n_seasons = data.season.hrcn.nunique(), bin_size = 5)
huracanpy.plot.density(d, 
                      cbar_kwargs=dict(label="Number of points per 5x5° box per year"),
                      )

In [ ]:
# Density of all points
d = gen.hrcn.get_density(n_seasons = data.season.hrcn.nunique(), bin_size = 10)
huracanpy.plot.density(d, 
                      cbar_kwargs=dict(label="Number of genesis per 10x10° box per year"),
                      )